# Distributed Tracing in Databricks with OpenTelemetry and Azure Monitor

In this notebook, we will set up distributed tracing and logging for a Databricks notebook. Distributed tracing allows us to monitor and visualize requests as they propagate through different components of a system. This can be useful to identify performance bottlenecks or trace errors.


## Setup

Removing all existing widgets and installing the needed traceparent widget. This cell is commented as the widget should already be present in this notebook, if not: uncomment and run it before you comment it again.

In [ ]:
# dbutils.widgets.removeAll()
# dbutils.widgets.text("traceparent", "00-80e1afed08e019fc1110464cfa66635c-7a085853722dc6d2-01")

Installation of python packages on the cluster.

In [ ]:
%pip install opentelemetry-semantic-conventions==0.38b0
%pip install opentelemetry-sdk==1.17.0
%pip install azure-monitor-opentelemetry-exporter==1.0.0b13

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Not uninstalling typing-extensions at /databricks/python

Restarting the kernel to pickup the latest packages and getting the correct tracecontext (the one passed on by the API call) from the widget.

In [ ]:
dbutils.library.restartPython()

In [ ]:
trace_context = dbutils.widgets.get("traceparent")

## Execution 

In [ ]:
from azure.monitor.opentelemetry.exporter import (
    AzureMonitorLogExporter,
    AzureMonitorTraceExporter
)

from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.trace import get_tracer_provider, set_tracer_provider, SpanKind
from opentelemetry.trace.propagation.tracecontext import TraceContextTextMapPropagator

from opentelemetry.sdk._logs import LoggerProvider, LoggingHandler
from opentelemetry.sdk._logs.export import BatchLogRecordProcessor
from opentelemetry._logs import get_logger_provider, set_logger_provider

from random import randrange

import logging
import time

def setup_tracing(connection_string, resource):
    tracer_provider = TracerProvider(resource=resource)
    set_tracer_provider(tracer_provider)
    trace_exporter = AzureMonitorTraceExporter(connection_string=connection_string)
    span_processor = BatchSpanProcessor(
        trace_exporter,
    )
    get_tracer_provider().add_span_processor(span_processor)

def setup_logging(connection_string, resource):
    logger_provider = LoggerProvider(resource=resource)
    set_logger_provider(logger_provider)
    log_exporter = AzureMonitorLogExporter(connection_string=connection_string)
    log_record_processor = BatchLogRecordProcessor(
        log_exporter
    )
    get_logger_provider().add_log_record_processor(log_record_processor)
    handler = LoggingHandler(logger_provider=get_logger_provider())
    logging.getLogger().addHandler(handler)

connection_string = "<connection_string_to_app_insights>"
resource = Resource.create({"service.name": "Databricks Notebook"})

setup_tracing(connection_string, resource)
setup_logging(connection_string, resource)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

logger.info("Inside notebook")
logger.info(f"Attempt to join an existing trace session with context: {trace_context}")

headers = {
    "traceparent": trace_context
}

print(headers)

# Explicitly the tracer is being created in here to ensure that we create it with the correct context from the headers.
context = TraceContextTextMapPropagator().extract(headers)
tracer = trace.get_tracer(__name__)
with tracer.start_as_current_span(name="Notebook Invoked", context=context, kind=SpanKind.SERVER) as span:
    print(span.get_span_context())
    with tracer.start_as_current_span(name='extract'):
        logger.info("Extract")
        time.sleep(randrange(start=3, stop=10))
    with tracer.start_as_current_span(name='transform'):
        logger.info("Transform")
        time.sleep(randrange(start=3, stop=10))
    with tracer.start_as_current_span(name='load'):
        logger.info("Load")
        time.sleep(randrange(start=3, stop=10))

{'traceparent': '00-80e1afed08e019fc1110464cfa66635c-7a085853722dc6d2-01'}
SpanContext(trace_id=0x80e1afed08e019fc1110464cfa66635c, span_id=0x987ff4e0704e9c48, trace_flags=0x01, trace_state=[], is_remote=False)
